In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
%matplotlib inline

import xgboost as xgb
from xgboost import plot_importance
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

data_p = pd.read_csv("train.csv")

#결손값 처리
data_p.isna().head(3)
print(data_p.isna().sum())


####0값을 검사할 피처명 리스트 객체 설정


#무응답 해당 피처 : education, engnat, hand, urban
zero_features = ['education', 'engnat', 'hand', 'urban']
    
#전체 데이터 건수
total_count = data_p['education'].count()

#피처별로 반복 하면서 데이터 값이 0인 데이터 건수 추출하고, 퍼센트 계산
for feature in zero_features:
    zero_count = data_p[data_p[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature,  zero_count, 100* zero_count/total_count))
    
    
Q_E = []


#Q_E 시간
for i in range(97, 117):
    a = 'Q' + chr(i) + 'E'
    Q_E.append(a)
    
print(Q_E)
    
for feature in Q_E:
    zero_count = data_p[data_p[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature,  zero_count, 100* zero_count/total_count))

    
    
#tp
TP = []
for i in range(1,10):
        a = 'tp0' + str(i)
        TP.append(a)
TP.append('tp10')
print(TP)

for feature in TP:
    zero_count = data_p[data_p[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature,  zero_count, 100* zero_count/total_count)) 
    
    
    
#0값을 평균값으로 대체
data_p[zero_features]=data_p[zero_features].replace(0, data_p[zero_features].mean())



#시간에 대해 이상치 데이터 인덱스 확인 및 제거

def get_outlier(df=None, column=None, weight=1.5):
    fraud = df[df['voted']==1][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    
    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    
    return outlier_index

for feature in Q_E:
    outlier_index = get_outlier(df=data_p, column=feature, weight=1.5)
    #print('이상치 데이터 인덱스 :', outlier_index)
    
data_p.drop(outlier_index, axis=0, inplace=True)



#데이터 변환
#원-핫 인코딩 대상 : gender, married, race, religion, 
#레이블 인코딩 대상 : age-group, 

from sklearn.preprocessing import LabelEncoder
data_p[''] = data_p['gender'] == ''


#age_group

age_group_=['10s', '20s', '30s', '40s', '50s', '60s', '+70s']

data_p['age_group_10s'] = data_p['gender'] == '10s'
data_p['age_group_20s'] = data_p['gender'] == '20s'
data_p['age_group_30s'] = data_p['gender'] == '30s'
data_p['age_group_40s'] = data_p['gender'] == '40s'
data_p['age_group_50s'] = data_p['gender'] == '50s'
data_p['age_group_60s'] = data_p['gender'] == '60s'
data_p['age_group_70s'] = data_p['gender'] == '70s'



from sklearn.preprocessing import OneHotEncoder

#gender

gender_=['Male', 'Female']

data_p['gender_Male'] = data_p['gender'] == 'Male'
data_p['gender_Female'] = data_p['gender'] == 'Female'

#marride

marride_=['1', '2', '3', '0']

data_p['married_1'] = data_p['married'] == 1
data_p['married_2'] = data_p['married'] == 2
data_p['married_3'] = data_p['married'] == 3
data_p['married_4'] = data_p['married'] == 4
    

#race

race_=['Asian', 'Arab', 'Black', 'Indigenous Australian', 'Native American', 'White', 'Other']

data_p['race_Asian'] = data_p['race'] == 'Asian'
data_p['race_Arab'] = data_p['race'] == 'Arab'
data_p['race_Black'] = data_p['race'] == 'Black'
data_p['race_Indigenous Australian'] = data_p['race'] == 'Indigenous Australian'
data_p['race_Native American'] = data_p['race'] == 'Native American'
data_p['race_White'] = data_p['race'] == 'White'
data_p['race_Other'] = data_p['race'] == 'Other'


#religion

religion_=['Agnostic', 'Atheist', 'Buddhist', 'Christian_Catholic', 'Christian_Mormon', 'Christian_Protestant', 'Christian_Other', 'Hindu', 'Jewish', 'Muslim', 'Sikh', 'Other']

data_p['religion_Agnostic'] = data_p['religion'] == 'Agnostic'
data_p['religion_Atheist'] = data_p['religion'] == 'Atheist'
data_p['religion_Buddhist'] = data_p['religion'] == 'Buddhist'
data_p['religion_Christian_Catholic'] = data_p['religion'] == 'Christian_Catholic'
data_p['religion_Christian_Mormon'] = data_p['religion'] == 'Christian_Mormon'
data_p['religion_Christian_Protestant'] = data_p['religion'] == 'Christian_Protestant'
data_p['religion_Christian_Other'] = data_p['religion'] == 'Christian_Other'
data_p['religion_Hindu'] = data_p['religion'] == 'Hindu'
data_p['religion_Jewish'] = data_p['religion'] == 'Jewish'
data_p['religion_Muslim'] = data_p['religion'] == 'Muslim'
data_p['religion_Sikh'] = data_p['religion'] == 'Sikh'
data_p['religion_Other'] = data_p['religion'] == 'Other'

data_p[['married_1','married_2','married_3','married_4','married','religion', 'gender', 'race']].head(30)


def dummy_data(data, columns) :
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data

dummy_columns = ['race', 'religion', 'gender', 'married']
data_k = dummy_data(data_p, dummy_columns)

data_k = data_k.drop('index', axis=1)
data_k = data_k.drop('age_group', axis=1)

data_k.head(20)


'''
X_features = data_k.iloc[:,:-1]
y_lables = data_k.iloc[:,-1]


print(X_features.shape)


X_train, X_test, y_train, y_test = train_test_split(X_features, y_lables, test_size =0.2, random_state=0, stratify=y_lables)

xgb_clf = XGBClassifier(n_estimators=500, random_state=156)

if len(feature_names) != len(set(feature_names)):
    raise ValueError('feature_names must be unique')

xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average='macro')

print(xgb_roc_score)
'''

index    0
QaA      0
QaE      0
QbA      0
QbE      0
        ..
wr_09    0
wr_10    0
wr_11    0
wr_12    0
wr_13    0
Length: 78, dtype: int64
education 0 건수는 432, 퍼센트는 1.19 %
engnat 0 건수는 63, 퍼센트는 0.17 %
hand 0 건수는 135, 퍼센트는 0.37 %
urban 0 건수는 258, 퍼센트는 0.71 %
['QaE', 'QbE', 'QcE', 'QdE', 'QeE', 'QfE', 'QgE', 'QhE', 'QiE', 'QjE', 'QkE', 'QlE', 'QmE', 'QnE', 'QoE', 'QpE', 'QqE', 'QrE', 'QsE', 'QtE']
QaE 0 건수는 0, 퍼센트는 0.00 %
QbE 0 건수는 0, 퍼센트는 0.00 %
QcE 0 건수는 0, 퍼센트는 0.00 %
QdE 0 건수는 0, 퍼센트는 0.00 %
QeE 0 건수는 0, 퍼센트는 0.00 %
QfE 0 건수는 0, 퍼센트는 0.00 %
QgE 0 건수는 0, 퍼센트는 0.00 %
QhE 0 건수는 1, 퍼센트는 0.00 %
QiE 0 건수는 1, 퍼센트는 0.00 %
QjE 0 건수는 1, 퍼센트는 0.00 %
QkE 0 건수는 2, 퍼센트는 0.01 %
QlE 0 건수는 0, 퍼센트는 0.00 %
QmE 0 건수는 0, 퍼센트는 0.00 %
QnE 0 건수는 0, 퍼센트는 0.00 %
QoE 0 건수는 0, 퍼센트는 0.00 %
QpE 0 건수는 1, 퍼센트는 0.00 %
QqE 0 건수는 1, 퍼센트는 0.00 %
QrE 0 건수는 0, 퍼센트는 0.00 %
QsE 0 건수는 0, 퍼센트는 0.00 %
QtE 0 건수는 0, 퍼센트는 0.00 %
['tp01', 'tp02', 'tp03', 'tp04', 'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10']
tp01 0 건수는 38

NameError: name 'feature_names' is not defined